In [1]:
import os
from tika import parser
import re
import pandas as pd
from bs4 import BeautifulSoup

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  


path = os.getcwd()


In [2]:
path

'C:\\Users\\luiz.cavalcante\\Desktop\\IR\\Luiz'

In [3]:
#path = r"C:\Users\luiz\Desktop\SERGIO"
path=path+"\\"

In [99]:
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))
historico = []
taxa =[]
for pdf in folders:
    a = parser.from_file(path+pdf)['content']
    a = re.sub('\n+', '\n', a)
    if 'NEGOCIAÇÃO' in a:
        a = re.sub('NEGOCIAÇÃO', 'CORRETAGEM', a)
        a = re.sub('Total Custos','Total corretagem', a)
        #print("ALERTA",pdf)
    folhas = re.findall(r"Data pregão(.*?)XP",a.replace('\n',''))
    aux = '_'.join(folhas[0].split('/')[::-1])
    #print(aux)
    os.rename(path+pdf,path+aux+".pdf")

In [100]:
folders = list(filter(lambda x: x[-4:]=='.pdf',os.listdir(path)))
historico = []
taxa =[]
for pdf in folders:
    a = parser.from_file(path+pdf)['content']
    a = re.sub('\n+', '\n', a)
    #########
    if 'NEGOCIAÇÃO' in a:
        a = re.sub('NEGOCIAÇÃO', 'CORRETAGEM', a)
        a = re.sub('Total Custos','Total corretagem', a)
    if 'WIN' in a:
        print("ALERTA WIN",pdf)
    if 'IND '  in a:
        print("ALERTA IND",pdf)
    ##########
    folhas = re.findall(r"NOTA DE CORRETAGEMNr. nota(.*?)Folha(.*?)Data pregão(.*?)XP",a.replace('\n',''))

    taxas = re.findall(r"Precatório\n([(0-9|.|,)]*)Total CBLC (D|C)\n([(0-9|.|,)]*)Valor líquido das operações (D|C)\n([(0-9|.|,)]*)Taxa de liquidação (D|C)\n([(0-9|.|,)]*)Taxa de Registro (D|C)\n([(0-9|.|,)]*)Total Bovespa / Soma (D|C)\n([(0-9|.|,)]*)Taxa de termo/opções (D|C)\n([(0-9|.|,)]*)Taxa A.N.A. (D|C)\n([(0-9|.|,)]*)Emolumentos (D|C)\n \n([(0-9|.|,)]*)Total corretagem / Despesas (D|C)\n \n([(0-9|.|,)]*)Taxa Operacional (D|C)\n([(0-9|.|,)]*)Execução\n([(0-9|.|,)]*)Taxa de Custódia\n([(0-9|.|,)]*)Impostos (D|C)\n([(0-9|.|,)]*)I.R.R.F. s/ operações, base R\$(.*?)\n([(0-9|.|,)]*)Outros (C|D)\n([(0-9|.|,)]*)Líquido para (.*?) (C|D)",a)
    #print(taxas)
    for i in range(len(taxas)):
        taxas[i] = list(taxas[i])
        if taxas[i][-6][-1] != "D":
            taxas[i][-7] = '0,00'
        assert taxas[i][20] == taxas[i][20] =='0,00'
        for j in range(15):
            if taxas[i][2*j+1][-1] == 'D':
                taxas[i][2*j] = '-'+taxas[i][2*j]
        if taxas[i][-1] == 'D':
                taxas[i][-3] = '-'+taxas[i][-3]
        taxas[i] = list(map(lambda x : int(x.replace('.','').replace(',','')) if not '/' in x else '-'.join(x.split('/')[::-1]),taxas[i][:-1:2] +[taxas[i][-2]]+[taxas[i][-6].split(' ')[0]]))
        try:
            lucro_daytrade,IRRF_daytrade = re.findall(r"IRRF Day Trade: Base R\$ (.*?) Projeção R\$ (.*?)\n",a)[0]
            taxas[i].append(-int(lucro_daytrade.replace('.','').replace(',','')))
            taxas[i].append(-int(IRRF_daytrade.replace('.','').replace(',','')))
        except:
            taxas[i].append(0)
            taxas[i].append(0)
        assert taxas[i][0] == sum(taxas[i][1:4])
        assert taxas[i][4] == sum(taxas[i][5:8])
        assert taxas[i][8] == sum(taxas[i][9:14])
        assert taxas[i][0] + taxas[i][4]+ taxas[i][8] + taxas[i][-1]== taxas[i][-5]
    #for k in taxas:
    #    print(k[-1],k[-5])
    #    print((k[1]*(1+0.00004105+0.0002795)+k[9]*(0.1068+0.039+1))/100 - k[-3]/100)

    trades = re.findall(r"Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/C(.*?)NOTA DE CORRETAGEM",a.replace('\n',''))
    trades2 = re.findall(r"CONTINUA...Corretagem / DespesasLíquido paraObservação: \(1\) As operações a termo não são computadas no líquido da fatura.Capitais e regiões metropolitanas: 4003-3710 Demais localidades: 0800-880-3710 SAC: 0800-772-0202 Ouvidoria: 0800-722-3710(.*?)Negócios realizadosQ Negociação C/V Tipo mercado Prazo Especificação do título Obs. \(\*\) Quantidade Preço / Ajuste Valor Operação / Ajuste D/CNOTA DE CORRETAGEM",a.replace('\n',''))

    assert trades.count('') == len(trades2)
    for i,j in enumerate(folhas):
        if j[1] != '1':
            assert trades[i] == ''
            trades[i] = trades2[0]
            trades2 = trades2[1:]


        aux = filter(lambda x: x !='',trades[i].split('1-'))
        aux2 = map(lambda x : re.findall(r"^(.*?) (C|V) (.*?) (.*?) ([(0-9|.|,)]*) ([(0-9|.|,)]*) ([(0-9|.|,)]*) (D|C)$",x)[0],aux)
        aux3 = map(lambda x : [x[0],x[1],x[2],re.sub(' +', ' ', x[3]).strip(),x[4].replace('.','').replace(',','.'),x[5].replace('.','').replace(',','.'),x[6].replace('.','').replace(',','.'),x[7]],aux2)
        folhas[i] = [j[0],j[1],'-'.join(j[2].split('/')[::-1]),list(aux3)] 

    assert len(trades2)==0

    folhas = folhas[::-1]
    for i,k in enumerate(folhas):
        #Aqui to perdendo a info do marcado D do daytrade no nome do ativo
        k[3] = list(map(lambda x : (k[0],k[2],re.sub(r'(#$| #$| #2$)','', x[3]),int('-'+x[4]),x[5],int(x[6].replace('.',''))) if x[1] == 'V' else (k[0],k[2],re.sub(r'(#$| #$| #2$)','', x[3]),int(x[4]),x[5],int('-'+x[6].replace('.',''))) ,k[3]))
        if i != 0:
            if folhas[i-1][1] != '1':
                folhas[i][3]= folhas[i][3]+folhas[i-1][3]
    folhas = list(filter(lambda x: x[1] == '1',folhas[::-1]))
    folhas = list(map(lambda x: x[3],folhas))
    assert len(folhas)==len(taxas)

    historico.append([n for m in folhas for n in m])
    taxa.append(taxas)
historico,taxa = [n for m in historico for n in m],[z for x in taxa for z in x]


ALERTA WIN 2019_07_15.pdf
ALERTA WIN 2019_12_30.pdf
ALERTA WIN 2020_01_31.pdf
ALERTA WIN 2020_02_28.pdf
ALERTA WIN 2020_03_31.pdf
ALERTA WIN 2020_04_06.pdf
ALERTA IND 2020_04_06.pdf
ALERTA WIN 2020_05_08.pdf
ALERTA WIN 2020_07_24.pdf


In [103]:

def translate_tickers(nome):
    tickers={
        'BTGP BANCO UNT N2':'BPAC11',
        'BTGP BANCO UNT N2 D':'BPAC11',
        'ENEVA ON NM':'ENEV3',
        'ENEVA ON NM D':'ENEV3',
        'BIOTOSCANA DR3':'GBIO33',
        'BIOTOSCANA DR3 D#2':'GBIO33',
        'BIOTOSCANA DR3 D':'GBIO33',
        'QUALICORP ON ER NM':'QUAL3',
        'QUALICORP ON NM':'QUAL3',
        'BRASIL ON NM #2D':'BBAS3',
        'BRASIL ON NM':'BBAS3',
        'BRASIL ON NM D':'BBAS3',
        'SINQIA ON NM':'SQIA3',
        'AES TIETE E UNT ED N2':'TIET11',
        'AES TIETE E UNT N2':'TIET11',
        'BRADESCO PN EJB N1':'BBDC4',
        'BRADESCO PN N1':'BBDC4',
        'BRADESCO ON EJB N1':'BBDC3',
        'BRADESCO ON N1':'BBDC3',
        'ITAUUNIBANCO PN ED N1':'ITUB4',
        'ITAUUNIBANCO PN N1':'ITUB4',
        'JBS ON ED NM':'JBSS3',
        'JBS ON NM':'JBSS3',
        'LOJAS RENNER ON EDB NM':'LREN3',
        'PETROBRAS PN N2 D':'PETR4',
        'LOJAS RENNER ON NM':'LREN3',
        'P.ACUCAR-CBD PN EJ N1':'PCAR4',
        'P.ACUCAR-CBD PN N1':'PCAR4',
        'PETROBRAS PN ED N2':'PETR4',
        'PETROBRAS PN N2':'PETR4',
        'BRASIL ON NM D':'BBAS3',
        'VALE ON NM':'VALE3',
        'AZUL PN N2':'AZUL4',
        'OI ON N1':'OIBR3',
        'OI ON N1 D':'OIBR3',
        'OI ON N1 D#2':'OIBR3',
        'B2W DIGITAL ON NM':'BTOW3',
        'ENGIE BRASIL ON NM':'EGIE3',
        'EQUATORIAL ON NM':'EQTL3',
        'LOCALIZA ON NM':'RENT3',
        'SUZANO PAPEL ON NM':'SUZB3',
        'CARREFOUR BR ON NM':'CRFB3',
        'TIM PART S/A ON NM':'TIMP3',
        'SUL AMERICA UNT N2':'SULA11',
        'MULTIPLAN ON N2':'MULT3',
        'TRAN PAULIST PN N1':'TRPL4',
        'RUMO S.A. ON NM':'RAIL3',
        'LOJAS RENNER ON ED NM D':'LREN3',
        'EZTEC ON NM':'EZTC3',
        'EZTEC ON EDB NM':'EZTC3',
        'B3 ON NM':'B3SA3',
        'ISHARES BOVA CI ATZ':'BOVA11',
        'LOCAWEB ON NM':'LWSA3',
        'RANDON PART PN N1':'RAPT4',
        'IGUATEMI ON NM' : 'IGTA3',
        'BR MALLS PAR ON NM':'BRML3',
        'MAGAZ LUIZA ON NM D':'MGLU3',
        'MAGAZ LUIZA ON NM':'MGLU3',
        'CVC BRASIL ON NM':'CVCB3',
        'TOTVS ON NM':'TOTS3',
        'KEPLER WEBER ON EJ':'KEPL3',
        'KEPLER WEBER ON':'KEPL3',
        'TUPY ON NM':'TUPY3',
        'PETRORIO ON NM':'PRIO3',
        'PETRORIO ON NM D':'PRIO3',
        'PETRORIO ON NM D#2':'PRIO3',
        'VIAVAREJO ON NM':'VVAR3',
        'CENTAURO ON NM':'CNTO3',
        'ELETROBRAS ON N1':'ELET3',
        'BRADESCO PN EJ N1':'BBDC4',
        'M.DIASBRANCO ON EJ NM':'MDIA3',
        'CIELO ON NM D' : 'CIEL3',
        'ABC BRASIL PN EJ N2':'ABCB4',
        'CAMIL ON NM':'CAML3',
        'GRUPO NATURA ON NM':'NTCO3',
        'CEA MODAS ON EJ NM':'CEAB3',
        'CEA MODAS ON NM':'CEAB3',
        'CIA HERING ON NM':'HGTX3',
        'IMC S/A ON NM':'MEAL3',
        'PPLA UNT':'PPLA11',
        'BANCO BMG PN N1':'BMGB4',
        'LOG-IN ON NM':'LOGN3',
        'DE COMPRA 01/20 VVARA125 ON 12,50 VVAR':'VVAR125',
        'OPC COMPRA 01/20 VVARA125E ON 12,50':'VVAR125',
        'PORTOBELLO ON NM':'PTBL3',
        'RECRUSUL PN':'RCSL4',
        'GAFISA ON NM':'GFSA3',
        'COGNA ON ON NM':'COGN3',
        'MILLS ON NM':'MILS3',
        'TECNISA ON NM':'TCSA3',
        'TRISUL ON NM' : 'TRIS3',
        'CEMIG PN N1' : 'CMIG4',
        'CEMIG PN EJ N1': 'CMIG4',
        'DOMMO ON':'DMMO3',
        'PETROBRAS BR ON NM':'BRDT3',
        'OI PN N1':'OIBR4',
        'ISHARES BOVA CI':'BOVA11',
        'ISHARES BOVA CI D':'BOVA11',
        'ELETROBRAS PNB N1':'ELET6',
        'FII XP CRED XPCI12 DM 100,51':'DSHFOSDHFOISDFJSDPOOFJSPOFJSDPF',
    }
    return tickers[nome]


In [104]:
aux = pd.DataFrame(historico,columns=['nota','pregao','nome','qtd','preco_medio','financeiro'])
aux['daytrade'] = aux.nome.apply(lambda x : True if x[-1] == 'D' or x[-3:-1] == 'D#' or x[-4:-1] == 'D#2' else False)
aux.nome = aux.nome.apply(translate_tickers)
aux['mes'] = aux['pregao'].apply(lambda x : '-'.join(x.split('-')[:2]))
aux = aux.sort_values(by='pregao',ascending=True)
aux.index = range(len(aux))
aux['turnover'] = abs(aux['financeiro'])



In [105]:
turnover = aux.groupby(by=['pregao','nota']).sum()[["turnover"]].sort_values(by='pregao',ascending=True)
turnover

,,turnover
pregao,nota,
2019-07-15,15472367,2131000
2019-08-02,15770099,2155400
2019-08-23,16129828,6909800
2019-08-26,16152489,5411200
2019-08-27,16172636,4751100
2019-08-29,16209057,452000
2019-09-25,16613152,1192500
2019-09-30,16668683,462000
2019-10-07,16779434,17710300


In [106]:
aux.groupby(by=['pregao','nota']).count().reset_index()[['pregao','nota']].sort_values(by='pregao',ascending=True)


,pregao,nota
0,2019-07-15,15472367
1,2019-08-02,15770099
2,2019-08-23,16129828
3,2019-08-26,16152489
4,2019-08-27,16172636
5,2019-08-29,16209057
6,2019-09-25,16613152
7,2019-09-30,16668683
8,2019-10-07,16779434
9,2019-10-18,16967257


In [107]:
df_taxas = pd.DataFrame(taxa).sort_values(by=15,ascending=True)
df_taxas.columns = ['CBLC','valor_liquido_operacoes','taxa_liquidacao','taxa_registro','total_bovespa','taxa_termo_opcoes','taxa_ANA','emolumentos','total_custos','taxa_operacional','execucao','impostos','IRRF','outros','valor_liquido_para','data_liquidacao','montante_IRRF','lucro_daytrade','IRRF_daytrade']
df_taxas.index = range(len(df_taxas))
#df_taxas = df_taxas[[15,14]]
#df_taxas.columns = ['liquidacao','IRRF','valorIRRF','corretagem','valor_final','valor_liquid_operacoes','lucro_daytrade','IRRF_daytrade']
#df_taxas['valor_liquido_para'] = df_taxas['valor_liquido_para'] - df_taxas['IRRF_daytrade']
df_taxas['custos_operacionais'] = df_taxas['taxa_liquidacao']+df_taxas['emolumentos']+df_taxas['impostos']+df_taxas['outros']+df_taxas['taxa_operacional']
df_taxas = pd.concat([df_taxas,aux.groupby(by=['pregao','nota']).count().reset_index()[['pregao','nota']].sort_values(by='pregao',ascending=True)],axis=1)
#df_taxas= df_taxas[['nota','pregao','data_liquidacao','taxa_operacional','custos_operacionais','valor_liquido_para']]
#df_taxas.set_index(15).T
df_taxas

,CBLC,valor_liquido_operacoes,taxa_liquidacao,taxa_registro,total_bovespa,taxa_termo_opcoes,taxa_ANA,emolumentos,total_custos,taxa_operacional,...,IRRF,outros,valor_liquido_para,data_liquidacao,montante_IRRF,lucro_daytrade,IRRF_daytrade,custos_operacionais,pregao,nota
0,-2131586,-2131000,-586,0,-89,0,0,-89,-4330,-3780,...,0,-147,-2136005,2019-07-17,0,0,0,-5005,2019-07-15,15472367
1,2154808,2155400,-592,0,-150,0,0,-150,-2271,-1890,...,-107,-73,2152387,2019-08-06,2155400,0,0,-2906,2019-08-02,15770099
2,-6911700,-6909800,-1900,0,-284,0,0,-284,-6496,-5670,...,0,-221,-6918480,2019-08-27,0,0,0,-8680,2019-08-23,16129828
3,-5412688,-5411200,-1488,0,-222,0,0,-222,-10827,-9450,...,0,-368,-5423737,2019-08-28,0,0,0,-12537,2019-08-26,16152489
4,-4752406,-4751100,-1306,0,-195,0,0,-195,-6496,-5670,...,0,-221,-4759097,2019-08-29,0,0,0,-7997,2019-08-27,16172636
5,451876,452000,-124,0,-18,0,0,-18,-2164,-1890,...,0,-73,449694,2019-09-02,452000,0,0,-2306,2019-08-29,16209057
6,-1192827,-1192500,-327,0,-83,0,0,-83,-2164,-1890,...,0,-73,-1195074,2019-09-27,0,0,0,-2574,2019-09-25,16613152
7,461873,462000,-127,0,-32,0,0,-32,-2164,-1890,...,0,-73,459677,2019-10-02,462000,0,0,-2323,2019-09-30,16668683
8,17705430,17710300,-4870,0,-727,0,0,-727,-3049,-1890,...,-885,-73,17701654,2019-10-09,17710300,0,0,-7761,2019-10-07,16779434
9,-2026157,-2025600,-557,0,-83,0,0,-83,-2164,-1890,...,0,-73,-2028404,2019-10-22,0,0,0,-2804,2019-10-18,16967257


In [138]:
#verifica se os daytrades estão todos marcados certo (soma por dia da coluna qtd tem q ser zero pra todos os ativos/dia)
e = aux[aux.daytrade].groupby(['pregao','nome']).sum()
e

,,qtd,financeiro,daytrade,turnover
pregao,nome,,,,
2019-12-06,BBAS3,-2798,13556310,2.0,13575690
2020-04-03,ENEV3,0,56000,19.0,13080000
2020-06-03,BOVA11,0,280,1.0,280
2020-06-15,BOVA11,0,14242,1.0,14242
2020-06-17,BOVA11,190,-1693730,11.0,16821310
2020-07-17,CIEL3,0,-2000,2.0,992000


In [141]:
#preciso na unha tirar os BOVA11 q na vdd n foram daytrade
confusao = aux[aux.pregao=='2020-06-17'][aux.nome == 'BOVA11']
confusao

C:\Users\luiz.cavalcante\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
403,22778460,2020-06-17,BOVA11,-110,91.13,1002430,True,2020-06,1002430
404,22778460,2020-06-17,BOVA11,30,90.76,-272280,False,2020-06,272280
405,22778460,2020-06-17,BOVA11,-50,91.13,455650,True,2020-06,455650
406,22778460,2020-06-17,BOVA11,-190,91.13,1731470,True,2020-06,1731470
407,22778460,2020-06-17,BOVA11,-210,91.13,1913730,True,2020-06,1913730
408,22778460,2020-06-17,BOVA11,-20,91.13,182260,True,2020-06,182260
409,22778460,2020-06-17,BOVA11,-40,91.13,364520,True,2020-06,364520
410,22778460,2020-06-17,BOVA11,420,90.76,-3811920,True,2020-06,3811920
411,22778460,2020-06-17,BOVA11,200,90.82,-1816400,False,2020-06,1816400
412,22778460,2020-06-17,BOVA11,400,90.76,-3630400,True,2020-06,3630400


In [142]:
qtd_comprada = confusao[confusao.qtd>0].qtd.sum()
qtd_vendida  = -confusao[confusao.qtd<0].qtd.sum()

#saldo final dos trades daquele dia, se ficou net comprado ou net vendido
saldo = qtd_comprada-qtd_vendida

#medio de compra
medio_compra = -0.01*confusao[confusao.qtd>0].financeiro.sum()/qtd_comprada
#medio_venda
medio_venda = 0.01*confusao[confusao.qtd<0].financeiro.sum()/qtd_vendida
medio_compra,medio_venda


(90.7768493150685, 91.13000000000001)

In [143]:
adicionar = confusao.iloc[0:2].copy()
adicionar.index = [0,1]
adicionar

,nota,pregao,nome,qtd,preco_medio,financeiro,daytrade,mes,turnover
0,22778460,2020-06-17,BOVA11,-110,91.13,1002430,True,2020-06,1002430
1,22778460,2020-06-17,BOVA11,30,90.76,-272280,False,2020-06,272280


In [144]:

if saldo > 0 and medio_compra < medio_venda :
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_compra
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    
elif saldo < 0 and medio_compra < medio_venda:
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_venda
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_comprada*(medio_venda-medio_compra)))
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_comprada*(medio_venda-medio_compra)))

elif saldo > 0 and medio_compra > medio_venda :
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_compra
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_vendida*(medio_venda-medio_compra))) 
    
    
elif saldo < 0 and medio_compra > medio_venda:
    adicionar.loc[0,'daytrade'] = False
    adicionar.loc[0,'qtd']  = saldo
    adicionar.loc[0,'preco_medio']  = medio_venda
    adicionar.loc[0,'financeiro']  = confusao.financeiro.sum() - int(round(100*qtd_comprada*(medio_venda-medio_compra)))
    
    adicionar.loc[1,'daytrade'] = True
    adicionar.loc[1,'qtd']  = 0
    adicionar.loc[1,'preco_medio']  = 0
    adicionar.loc[1,'financeiro']  = int(round(100*qtd_comprada*(medio_venda-medio_compra)))

    
else:
    print('algum erro de caso inesperado')


#adicionar.index = [len(aux),len(aux)+1]
adicionar['turnover']  = adicionar['financeiro'].apply(abs)

In [145]:
aux.drop(confusao.index,inplace=True)
aux = pd.concat([aux,adicionar]).sort_values('pregao')
aux.index =range(len(aux))

In [146]:
#agora todos os daytrades devem estar todos marcados certo (soma por dia da coluna qtd tem q ser zero pra todos os ativos/dia)
e = aux[aux.daytrade].groupby(['pregao','nome']).sum()
e

,,qtd,financeiro,daytrade,turnover
pregao,nome,,,,
2019-12-06,BBAS3,-2798,13556310,2.0,13575690
2020-04-03,ENEV3,0,56000,19.0,13080000
2020-06-03,BOVA11,0,280,1.0,280
2020-06-15,BOVA11,0,14242,1.0,14242
2020-06-17,BOVA11,0,29312,1.0,29312
2020-07-17,CIEL3,0,-2000,2.0,992000


In [151]:
#Quantidade de dinheiro ganhado no daytrade BVM (SEM INDICIE FUTURO) (ainda não estao incluidos os custos)
e['financeiro'].sum()/100

136541.44

In [152]:
taxas_detalhadas = df_taxas[['nota','pregao','data_liquidacao','valor_liquido_operacoes','custos_operacionais','valor_liquido_para','IRRF','IRRF_daytrade']]
taxas_detalhadas

,nota,pregao,data_liquidacao,valor_liquido_operacoes,custos_operacionais,valor_liquido_para,IRRF,IRRF_daytrade
0,15472367,2019-07-15,2019-07-17,-2131000,-5005,-2136005,0,0
1,15770099,2019-08-02,2019-08-06,2155400,-2906,2152387,-107,0
2,16129828,2019-08-23,2019-08-27,-6909800,-8680,-6918480,0,0
3,16152489,2019-08-26,2019-08-28,-5411200,-12537,-5423737,0,0
4,16172636,2019-08-27,2019-08-29,-4751100,-7997,-4759097,0,0
5,16209057,2019-08-29,2019-09-02,452000,-2306,449694,0,0
6,16613152,2019-09-25,2019-09-27,-1192500,-2574,-1195074,0,0
7,16668683,2019-09-30,2019-10-02,462000,-2323,459677,0,0
8,16779434,2019-10-07,2019-10-09,17710300,-7761,17701654,-885,0
9,16967257,2019-10-18,2019-10-22,-2025600,-2804,-2028404,0,0


In [153]:
assert (df_taxas.valor_liquido_operacoes+df_taxas.custos_operacionais+df_taxas.IRRF+df_taxas.IRRF_daytrade == df_taxas.valor_liquido_para).all()
assert (aux.groupby(['nota']).sum().financeiro.values == taxas_detalhadas.valor_liquido_operacoes.values).all()

In [154]:
df_taxas[['nota','pregao','data_liquidacao','valor_liquido_operacoes','custos_operacionais','valor_liquido_para','IRRF','IRRF_daytrade']]

,nota,pregao,data_liquidacao,valor_liquido_operacoes,custos_operacionais,valor_liquido_para,IRRF,IRRF_daytrade
0,15472367,2019-07-15,2019-07-17,-2131000,-5005,-2136005,0,0
1,15770099,2019-08-02,2019-08-06,2155400,-2906,2152387,-107,0
2,16129828,2019-08-23,2019-08-27,-6909800,-8680,-6918480,0,0
3,16152489,2019-08-26,2019-08-28,-5411200,-12537,-5423737,0,0
4,16172636,2019-08-27,2019-08-29,-4751100,-7997,-4759097,0,0
5,16209057,2019-08-29,2019-09-02,452000,-2306,449694,0,0
6,16613152,2019-09-25,2019-09-27,-1192500,-2574,-1195074,0,0
7,16668683,2019-09-30,2019-10-02,462000,-2323,459677,0,0
8,16779434,2019-10-07,2019-10-09,17710300,-7761,17701654,-885,0
9,16967257,2019-10-18,2019-10-22,-2025600,-2804,-2028404,0,0


In [155]:
df_taxas.IRRF.sum()/100,df_taxas.IRRF_daytrade.sum()/100,taxas_detalhadas[['IRRF','IRRF_daytrade']].sum().sum()/100

(-136.37, -8.29, -144.66)

In [157]:
teste = aux.groupby(['nota','pregao','mes','nome','daytrade']).sum().reset_index()
teste

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover
0,15472367,2019-07-15,2019-07,TRIS3,False,2600,-2131000,2131000
1,15770099,2019-08-02,2019-08,TRIS3,False,-2600,2155400,2155400
2,16129828,2019-08-23,2019-08,GBIO33,False,7900,-6909800,6909800
3,16152489,2019-08-26,2019-08,GBIO33,False,6200,-5411200,5411200
4,16172636,2019-08-27,2019-08,GBIO33,False,5200,-4751100,4751100
...,...,...,...,...,...,...,...,...
74,23913583,2020-07-20,2020-07,BOVA11,False,-830,8277590,8277590
75,23913583,2020-07-20,2020-07,OIBR3,False,66000,-9012000,9012000
76,24156262,2020-07-24,2020-07,OIBR3,False,22800,-3830400,3830400
77,24156262,2020-07-24,2020-07,VVAR3,False,-2800,5437600,5437600


In [159]:
#valor em REAIS que foi cobrado a mais no 'taxa_liquidacoo'
print((-df_taxas.taxa_liquidacao-turnover.reset_index().turnover*0.0002795).sum()/100)
(-df_taxas.taxa_liquidacao-turnover.reset_index().turnover*0.0002795)/100

-65.53258660000012


0     -0.096145
1     -0.104343
2     -0.312891
3     -0.244304
4     -0.219325
5     -0.023340
6     -0.063038
7     -0.021290
8     -0.800289
9     -0.091552
10    -0.588943
11    -0.092054
12    -0.628513
13    -0.055552
14    -0.064620
15    -0.067720
16    -0.082931
17    -0.020672
18    -0.644054
19    -1.459865
20    -0.948449
21    -0.154029
22    -0.248214
23    -0.570120
24    -1.247992
25    -0.923900
26    -1.385196
27    -0.167841
28    -1.053524
29    -3.676531
30    -2.351610
31    -1.654578
32    -1.757235
33    -1.674000
34    -0.296167
35   -11.572325
36    -0.368385
37    -0.362549
38    -0.362392
39    -0.324006
40    -0.200829
41    -0.106700
42    -1.744731
43    -0.995775
44    -9.215966
45    -0.264600
46   -12.264102
47    -0.298990
48    -0.451649
49    -0.517950
50    -0.792640
51    -0.784404
52    -0.424060
53    -0.689712
dtype: float64

In [160]:
#valor em REAIS que foi cobrado a mais no 'emolumento'
print((-df_taxas.emolumentos-turnover.reset_index().turnover*0.00004105).sum()/100)
(-df_taxas.emolumentos-turnover.reset_index().turnover*0.00004105)/100

35.00233745999999


0      0.015224
1      0.615208
2      0.003527
3     -0.001298
4     -0.000327
5     -0.005546
6      0.340479
7      0.130349
8     -0.000078
9     -0.001509
10    -0.007641
11    -0.004023
12    -0.008315
13    -0.003109
14    -0.007378
15    -0.001268
16    -0.005749
17    -0.018437
18    -0.422821
19    -0.988444
20     5.186618
21    -0.107384
22    -0.251527
23    -0.563384
24    -1.240054
25    -0.912410
26    -1.367752
27    -0.164878
28    -1.040528
29    23.592189
30    10.519141
31    -2.359538
32    -3.353047
33    10.769400
34     1.856914
35    -4.304918
36     2.244769
37     2.287937
38     2.308586
39    -0.776291
40    -0.331086
41    -0.177730
42    -3.157278
43    -0.833473
44    -1.686148
45     1.696260
46    -1.705465
47    -0.667281
48     0.978538
49     3.302230
50    -0.107216
51    -1.757377
52    -0.944514
53    -1.559813
dtype: float64

In [161]:
#valor em centavos que foi cobrado a mais no 'imposto'
-df_taxas.impostos+(0.1068)*df_taxas.reset_index()['taxa_operacional']

0    -0.704
1    -0.852
2    -0.556
3    -0.260
4    -0.556
5    -0.852
6    -0.852
7    -0.852
8    -0.852
9    -0.852
10   -0.852
11   -0.852
12   -0.852
13   -0.852
14   -0.852
15   -0.852
16   -0.852
17   -0.852
18   -0.704
19   -0.556
20   -0.556
21   -0.852
22   -0.556
23   -0.556
24   -0.852
25   -0.852
26   -0.408
27   -0.704
28   -0.556
29   -0.704
30   -0.704
31   -0.852
32   -0.556
33   -0.852
34   -0.852
35   -0.876
36   -0.704
37   -0.852
38   -0.852
39   -0.704
40   -0.852
41   -0.852
42   -0.556
43   -0.292
44   -0.024
45   -0.852
46   -0.864
47   -0.704
48   -0.704
49   -0.704
50   -0.880
51   -0.408
52   -0.704
53   -0.852
dtype: float64

In [162]:
#valor em centavos que foi cobrado a mais na 'outros'
-df_taxas.outros+(0.039)*df_taxas.reset_index()['taxa_operacional']

0    -0.42
1    -0.71
2    -0.13
3    -0.55
4    -0.13
5    -0.71
6    -0.71
7    -0.71
8    -0.71
9    -0.71
10   -0.71
11   -0.71
12   -0.71
13   -0.71
14   -0.71
15   -0.71
16   -0.71
17   -0.71
18   -0.42
19   -0.13
20   -0.13
21   -0.71
22   -0.13
23   -0.13
24   -0.71
25   -0.71
26   -0.84
27   -0.42
28   -0.13
29   -0.42
30   -0.42
31   -0.71
32   -0.13
33   -0.71
34   -0.71
35   -0.73
36   -0.42
37   -0.71
38   -0.71
39   -0.42
40   -0.71
41   -0.71
42   -0.13
43   -0.91
44   -0.02
45   -0.71
46   -0.22
47   -0.42
48   -0.42
49   -0.42
50   -0.40
51   -0.84
52   -0.42
53   -0.71
dtype: float64

In [163]:
#taxa_liquidacoo
-turnover.reset_index().turnover*0.0002795/100
#emolumentos
-turnover.reset_index().turnover*0.00004105/100
#impostos
(0.1068)*df_taxas.reset_index()['taxa_operacional']/100
#outros
(0.039)*df_taxas.reset_index()['taxa_operacional']/100

0    -1.4742
1    -0.7371
2    -2.2113
3    -3.6855
4    -2.2113
5    -0.7371
6    -0.7371
7    -0.7371
8    -0.7371
9    -0.7371
10   -0.7371
11   -0.7371
12   -0.7371
13   -0.7371
14   -0.7371
15   -0.7371
16   -0.7371
17   -0.7371
18   -1.4742
19   -2.2113
20   -2.2113
21   -0.7371
22   -2.2113
23   -2.2113
24   -0.7371
25   -0.7371
26   -2.9484
27   -1.4742
28   -2.2113
29   -1.4742
30   -1.4742
31   -0.7371
32   -2.2113
33   -0.7371
34   -0.7371
35   -3.1473
36   -1.4742
37   -0.7371
38   -0.7371
39   -1.4742
40   -0.7371
41   -0.7371
42   -2.2113
43   -1.0491
44   -2.4102
45   -0.7371
46   -1.9422
47   -1.4742
48   -1.4742
49   -1.4742
50   -0.6240
51   -2.9484
52   -1.4742
53   -0.7371
Name: taxa_operacional, dtype: float64

In [164]:
custo_distribuido = aux.groupby(['nota','pregao','mes','nome','daytrade']).sum().reset_index()
custo_distribuido['custo'] = 0
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo
0,15472367,2019-07-15,2019-07,TRIS3,False,2600,-2131000,2131000,0
1,15770099,2019-08-02,2019-08,TRIS3,False,-2600,2155400,2155400,0
2,16129828,2019-08-23,2019-08,GBIO33,False,7900,-6909800,6909800,0
3,16152489,2019-08-26,2019-08,GBIO33,False,6200,-5411200,5411200,0
4,16172636,2019-08-27,2019-08,GBIO33,False,5200,-4751100,4751100,0
...,...,...,...,...,...,...,...,...,...
74,23913583,2020-07-20,2020-07,BOVA11,False,-830,8277590,8277590,0
75,23913583,2020-07-20,2020-07,OIBR3,False,66000,-9012000,9012000,0
76,24156262,2020-07-24,2020-07,OIBR3,False,22800,-3830400,3830400,0
77,24156262,2020-07-24,2020-07,VVAR3,False,-2800,5437600,5437600,0


In [165]:
for nota in set(teste.nota):
    constante = custo_distribuido[custo_distribuido.nota == nota].turnover.sum()
    custo_total = taxas_detalhadas[taxas_detalhadas.nota == nota].custos_operacionais.sum()
    fator = custo_total/constante
    custo_distribuido['custo'] = custo_distribuido['custo']+ custo_distribuido.nota.apply(lambda x : fator if x==nota else 0)*custo_distribuido.turnover

In [166]:
#valor infimo
custo_distribuido.groupby('nota')[['custo']].sum().values - taxas_detalhadas.groupby('nota')[['custos_operacionais']].sum().values

array([[ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 4.54747351e-13],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [-1.81898940e-12],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.00000000e+00],
       [ 0.0

In [167]:
custo_distribuido = custo_distribuido.merge(taxas_detalhadas[['nota','data_liquidacao']],left_on='nota', right_on='nota')
custo_distribuido.head(6)

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao
0,15472367,2019-07-15,2019-07,TRIS3,False,2600,-2131000,2131000,-5005.0,2019-07-17
1,15770099,2019-08-02,2019-08,TRIS3,False,-2600,2155400,2155400,-2906.0,2019-08-06
2,16129828,2019-08-23,2019-08,GBIO33,False,7900,-6909800,6909800,-8680.0,2019-08-27
3,16152489,2019-08-26,2019-08,GBIO33,False,6200,-5411200,5411200,-12537.0,2019-08-28
4,16172636,2019-08-27,2019-08,GBIO33,False,5200,-4751100,4751100,-7997.0,2019-08-29
5,16209057,2019-08-29,2019-08,GBIO33,False,-500,452000,452000,-2306.0,2019-09-02


In [168]:
custo_distribuido['aux'] = custo_distribuido['financeiro'] +custo_distribuido['custo']
custo_distribuido['aux']=custo_distribuido['aux'].apply(int)/100
custo_distribuido[['pregao','data_liquidacao','nome','daytrade','aux']]

,pregao,data_liquidacao,nome,daytrade,aux
0,2019-07-15,2019-07-17,TRIS3,False,-21360.05
1,2019-08-02,2019-08-06,TRIS3,False,21524.94
2,2019-08-23,2019-08-27,GBIO33,False,-69184.80
3,2019-08-26,2019-08-28,GBIO33,False,-54237.37
4,2019-08-27,2019-08-29,GBIO33,False,-47590.97
...,...,...,...,...,...
74,2020-07-20,2020-07-22,BOVA11,False,82709.11
75,2020-07-20,2020-07-22,OIBR3,False,-90192.70
76,2020-07-24,2020-07-28,OIBR3,False,-38333.60
77,2020-07-24,2020-07-28,VVAR3,False,54333.96


In [182]:
custo_distribuido.groupby('nome').sum()[['aux']]

,aux
nome,
BBAS3,135477.35
BOVA11,274333.42
BRML3,52.96
CIEL3,-40.60
CMIG4,-7979.50
CNTO3,-7180.00
COGN3,616.83
CVCB3,-18664.95
ENEV3,492.47


In [183]:
#=IFERROR(DATE(YEAR(A2),DAY(A2),MONTH(A2)),DATE(INT(RIGHT(A2,4)),INT(MID(A2,4,2)),INT(LEFT(A2,2))))
#=IF(ISNUMBER(SEARCH("MARGEM",C2)),D2,0)
#custo_distribuido.loc[(1,3,28,29,30),:].groupby('nome').sum()[['qtd']]
#custo_distribuido.loc[(set(custo_distribuido.index)-{1,3,28,29,30}),:][~custo_distribuido.daytrade].aux.sum()
#custo_distribuido[custo_distribuido.nome=='oibr3'.upper()][['pregao','data_liquidacao','nome','daytrade','aux']]

In [180]:
custo_distribuido.groupby('nome').sum()[['qtd']]

,qtd
nome,
BBAS3,-2798
BOVA11,-2290
BRML3,0
CIEL3,0
CMIG4,0
CNTO3,0
COGN3,0
CVCB3,0
ENEV3,0


In [178]:
print(custo_distribuido[custo_distribuido.nome=='PRIO3'].aux.sum())
custo_distribuido[custo_distribuido.nome=='PRIO3']

-874.2799999998533


,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
30,18513199,2020-01-14,2020-01,PRIO3,False,5000,-20420000,20420000,-8526.000000,2020-01-16,-204285.26
33,18545780,2020-01-15,2020-01,PRIO3,False,4400,-18498900,18498900,-10984.641130,2020-01-17,-185098.84
35,18579106,2020-01-16,2020-01,PRIO3,False,400,-1668800,1668800,-2572.089323,2020-01-20,-16713.72
37,18780954,2020-01-24,2020-01,PRIO3,False,2500,-11670000,11670000,-6887.578107,2020-01-28,-116768.87
39,18904168,2020-01-30,2020-01,PRIO3,False,-12300,54120000,54120000,-21545.737428,2020-02-03,540984.54
41,18934102,2020-01-31,2020-01,PRIO3,False,8200,-38458000,38458000,-16122.601097,2020-02-04,-384741.22
42,19159690,2020-02-11,2020-02,PRIO3,False,-8200,36588400,36588400,-13491.000000,2020-02-13,365749.09


In [175]:
custo_distribuido[custo_distribuido.daytrade].groupby(['pregao','nome']).sum()

,,daytrade,qtd,financeiro,turnover,custo,aux
pregao,nome,,,,,,
2019-12-06,BBAS3,True,-2798,13556310,13575690,-8575.000000,135477.35
2020-04-03,ENEV3,True,0,56000,13080000,-6752.067459,492.47
2020-06-03,BOVA11,True,0,280,280,-0.183644,2.79
2020-06-15,BOVA11,True,0,14242,14242,-18.728066,142.23
2020-06-17,BOVA11,True,0,29312,29312,-55.995276,292.56
2020-07-17,CIEL3,True,0,-2000,992000,-2060.000000,-40.60


In [182]:
(24.09*98700+custo_distribuido[custo_distribuido.nome=='SQIA3'].aux.sum())*15/100


49052.765999999996

In [173]:
-custo_distribuido[custo_distribuido.nome=='SQIA3'].aux.sum()

2050664.56

In [174]:
custo_distribuido[~custo_distribuido.daytrade][custo_distribuido.nome!='SQIA3'].aux.sum()

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


351459.52

In [165]:
custo_distribuido[custo_distribuido.daytrade].aux.sum()

26152.139999999996

In [104]:
custo_distribuido[custo_distribuido.nome=='oibr3'.upper()][['pregao','data_liquidacao','nome','daytrade','aux']]

,pregao,data_liquidacao,nome,daytrade,aux
16,2020-05-04,2020-05-06,OIBR3,True,31251.10
35,2020-06-15,2020-06-17,OIBR3,False,-1019030.78
40,2020-06-18,2020-06-22,OIBR3,False,1096598.12
41,2020-06-23,2020-06-25,OIBR3,False,-514059.86
42,2020-06-26,2020-06-30,OIBR3,False,-232093.07
45,2020-07-24,2020-07-28,OIBR3,False,-50437.04
46,2020-07-31,2020-08-04,OIBR3,False,1184096.06


In [105]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,-21518.000000,2020-04-08,3035.82
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,-11253.000000,2020-04-09,-156105.53
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,-23216.156106,2020-04-13,7467.83
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,-3017.344067,2020-04-13,-99780.17
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,-10441.976662,2020-04-13,-345304.41
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,-18305.523165,2020-04-13,-1263.05
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400,-12393.000000,2020-04-14,335210.07
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000,-20889.748868,2020-04-20,-606508.89
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000,-33799.841068,2020-04-20,-981337.99
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000,-16507.139506,2020-04-20,-479265.07


In [46]:
custo_distribuido[custo_distribuido.nome=='SQIA3']

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
20,17948067,2019-12-13,2019-12,SQIA3,False,10600,-22857000,22857000,-11744.005317,2019-12-17,-228687.44
24,18218512,2019-12-30,2019-12,SQIA3,False,1300,-3282300,3282300,-3190.000000,2020-01-03,-32854.90
26,18314101,2020-01-06,2020-01,SQIA3,False,-600,1498500,1498500,-2233.660423,2020-01-08,14962.66
29,18480725,2020-01-13,2020-01,SQIA3,False,-11300,27673700,27673700,-10786.000000,2020-01-15,276629.14


In [47]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,15472367,2019-07-15,2019-07,TRIS3,False,2600,-2131000,2131000,-5005.000000,2019-07-17,-21360.05
1,15770099,2019-08-02,2019-08,TRIS3,False,-2600,2155400,2155400,-2906.000000,2019-08-06,21524.94
2,16129828,2019-08-23,2019-08,GBIO33,False,7900,-6909800,6909800,-8680.000000,2019-08-27,-69184.80
3,16152489,2019-08-26,2019-08,GBIO33,False,6200,-5411200,5411200,-12537.000000,2019-08-28,-54237.37
4,16172636,2019-08-27,2019-08,GBIO33,False,5200,-4751100,4751100,-7997.000000,2019-08-29,-47590.97
5,16209057,2019-08-29,2019-08,GBIO33,False,-500,452000,452000,-2306.000000,2019-09-02,4496.94
6,16613152,2019-09-25,2019-09,GBIO33,False,1500,-1192500,1192500,-2574.000000,2019-09-27,-11950.74
7,16668683,2019-09-30,2019-09,GBIO33,False,-600,462000,462000,-2323.000000,2019-10-02,4596.77
8,16779434,2019-10-07,2019-10,GBIO33,False,-19700,17710300,17710300,-7761.000000,2019-10-09,177025.39
9,16967257,2019-10-18,2019-10,QUAL3,False,600,-2025600,2025600,-2804.000000,2019-10-22,-20284.04


,qtd
nome,
BBAS3,-2798
BOVA11,-2290
BRML3,0
CIEL3,0
CMIG4,0
CNTO3,0
COGN3,0
CVCB3,0
ENEV3,0


In [108]:
qtd = aux.groupby(by=['pregao','nome']).sum()[["qtd"]].reset_index().pivot(index='nome',columns='pregao', values='qtd').fillna(0)
qtd

pregao,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-16,2020-04-24,2020-04-29,2020-04-30,2020-05-04,2020-05-06,...,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-23,2020-06-26,2020-07-02,2020-07-08,2020-07-24,2020-07-31
nome,,,,,,,,,,,,,,,,,,,,,
BOVA11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-7890.0,0.0,3740.0,4150.0,0.0,0.0,0.0,0.0,0.0,0.0
BPAC11,0.0,0.0,0.0,0.0,15000.0,0.0,0.0,0.0,0.0,0.0,...,-1000.0,-5900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNTO3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3200.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COGN3,0.0,0.0,0.0,0.0,180000.0,0.0,-180000.0,40000.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENEV3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GBIO33,0.0,16400.0,10500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-26900.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LWSA3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1100.0,-1100.0,0.0,0.0
MGLU3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OIBR3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1050000.0,0.0,0.0,-1050000.0,443000.0,200000.0,0.0,0.0,30000.0,-673000.0


In [109]:
financeiro = aux.groupby(by=['pregao','nome']).sum()[["financeiro"]].reset_index().pivot(index='nome',columns='pregao', values='financeiro').fillna(0)
financeiro/100

pregao,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-16,2020-04-24,2020-04-29,2020-04-30,2020-05-04,2020-05-06,...,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-23,2020-06-26,2020-07-02,2020-07-08,2020-07-24,2020-07-31
nome,,,,,,,,,,,,,,,,,,,,,
BOVA11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,702155.8,0.0,-340078.2,-385155.0,0.0,0.0,0.0,0.0,0.0,0.0
BPAC11,0.0,0.0,7700.0,0.0,-606300.0,0.0,0.0,0.0,0.0,0.0,...,61360.0,378852.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CNTO3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-101120.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
COGN3,0.0,0.0,0.0,0.0,-981000.0,0.0,1006200.0,-221600.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENEV3,3251.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
GBIO33,0.0,-155993.0,-99750.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,268731.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
LWSA3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,-55253.0,49951.0,0.0,0.0
MGLU3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
OIBR3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31746.0,0.0,...,-1018500.0,0.0,0.0,1097000.0,-513880.0,-232000.0,0.0,0.0,-50400.0,1184480.0


In [110]:
custo_distribuido

,nota,pregao,mes,nome,daytrade,qtd,financeiro,turnover,custo,data_liquidacao,aux
0,20630020,2020-04-06,2020-04,ENEV3,True,0,325100,59671100,-21518.000000,2020-04-08,3035.82
1,20665692,2020-04-07,2020-04,GBIO33,False,16400,-15599300,15599300,-11253.000000,2020-04-09,-156105.53
2,20702999,2020-04-08,2020-04,BPAC11,True,0,770000,76750000,-23216.156106,2020-04-13,7467.83
3,20702999,2020-04-08,2020-04,GBIO33,False,10500,-9975000,9975000,-3017.344067,2020-04-13,-99780.17
4,20702999,2020-04-08,2020-04,PETR4,False,20000,-34520000,34520000,-10441.976662,2020-04-13,-345304.41
5,20702999,2020-04-08,2020-04,PETR4,True,0,-108000,60516000,-18305.523165,2020-04-13,-1263.05
6,20739679,2020-04-09,2020-04,PETR4,False,-20000,33533400,33533400,-12393.000000,2020-04-14,335210.07
7,20888575,2020-04-16,2020-04,BPAC11,False,15000,-60630000,60630000,-20889.748868,2020-04-20,-606508.89
8,20888575,2020-04-16,2020-04,COGN3,False,180000,-98100000,98100000,-33799.841068,2020-04-20,-981337.99
9,20888575,2020-04-16,2020-04,TIET11,False,30000,-47910000,47910000,-16507.139506,2020-04-20,-479265.07


In [621]:
custo_distribuido[custo_distribuido.mes.isin(['2020-04','2020-05'])][custo_distribuido.daytrade].financeiro.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


28407.0

In [623]:
custo_distribuido[custo_distribuido.mes.isin(['2020-04','2020-05'])][custo_distribuido.daytrade].custo.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-2228.6067927113077

In [635]:
#Imposto de renda relativo a daytrade pra ser pago no final do mes 6,referente ao mês 5
(28407.0-2228.6067927113077-12664.86 -12828.95 )*20/100 + taxas_detalhadas.IRRF_daytrade.sum()/100




-268.4633585422617

In [652]:
custo_distribuido[~custo_distribuido.daytrade][~custo_distribuido.nome.isin(['OIBR3','SQIA3'])].financeiro.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-109637.65

In [650]:
custo_distribuido[~custo_distribuido.daytrade][~custo_distribuido.nome.isin(['OIBR3','SQIA3'])].custo.sum()/100

c:\program files\python36\lib\site-packages\ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


-3976.2850533131127